# An executable notebook
This notebook can also be converted to a script with 

    jupyter nbconvert --to script multiproc.ipynb
    
and then executed with

    python multiproc.py [ARGS]
    
If ipython magic is used, it need to be called not with `python` but with `ipython`, but then arguments cause issues.

In [16]:
import argparse
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
from multiprocessing import Pool
import numpy as np
import matplotlib.pyplot as plt
import shutil
import pickle
import os
import sys
import dustpy
#import pandas as pd
from dustpy.sim.utils import bindFunction
from scipy  import interpolate
#from widget import plotter
# to find out if we are running interactively
import __main__
is_interactive = not hasattr(__main__, '__file__')

In [17]:
import pickle
class Extra_var(object):
    def __init__(self, name,Mp1=None,Mp2=None,Mp3 = None,rp1=None,rp2=None,rp3=None,alpha =None,gamma = None,Mgas = None,rc=None ,tp2 =None,tp3 = None,):
        self.name = name
        self.Mp1 =  Mp1
        self.Mp2 =  Mp2
        self.Mp3 =  Mp3
        self.rp1 = rp1
        self.rp2 = rp2
        self.rp3 = rp3
        self.alpha = alpha
        self.gamma = gamma
        self.Mgas  = Mgas
        self.tp2 = tp2
        self.tp3 =  tp3
        self.rc = rc
def save_object(obj, filename):
       pickle_out = open(filename, 'wb')   # Overwrites any existing file.
       pickle.dump(obj, pickle_out)
       pickle_out.close()

# define the parser
parser = argparse.ArgumentParser(description='This python notebook is executable',formatter_class=argparse.RawTextHelpFormatter)

parser.add_argument('-o','--opacity',  help='what opacity to use'    ,    type=str, default='ricci', choices=['ricci','dsharp'])
parser.add_argument('-n','--number' ,  help='which model to run'     ,    type=int, default=-1)
parser.add_argument('-c','--cpus'   ,  help='how many cores to use'  ,    type=int, default=4 )

parser.description ='Here we can put more description'
parser.description+='And even more if necessary'
# use parser default if interactive
if not is_interactive:
    print('not interactive, will parse arguments')
    argsin = parser.parse_args()
else:
    print('running interactively, will use some defaults')
   
    argsin = parser.parse_args(['-o','dsharp'])
    
print('we are using the opacity from {}'.format(argsin.opacity))

running interactively, will use some defaults
we are using the opacity from dsharp


Define a worker function that takes some time to run

In [20]:
from importlib import reload
import datetime
now = datetime.datetime.now()
from functions_restart import loadSimulation

def Restart_function(nameSim):
    
    """ Function for restarting a given simulation. It works like the notebook bestFit.ipynb, but here the only argument
    is the simulation name"""
    
    from dustpy.sim.utils import readFromDump
    d = readFromDump(str(nameSim)+'/dustpy.dmp') 
      
    #d.snapshots = np.linspace(2,15*1e6,60)*c.yr

    def initialGas(A,r):
        
        return kanagawa_time_3gap(A,r,0)
    
    """ As some variables are not stored automathically in dmp file, those have to be uploaded from Extra_vars and  be set up properly."""
    def updateGas(d):
        
        with open(str(nameSim)+'/Extra_vars.pickle', 'rb') as F:
             P = pickle.load(F)
             
        d.gas.Sigma = kanagawa_time_3gap(d,d.grid.r,t=d.t,Mp1=P.Mp1,Mp2=P.Mp2,Mp3 = P.Mp3,tp2 = P.tp2,tp3 = P.tp3,Mgas = P.Mgas,gamma = P.gamma,rp1 =P.rp1,rp2 =P.rp2, rp3 = P.rp3,rc = P.rc,profile=P.profile,alpha =P.alpha)
    def updateAlpha(A):
        return np.array([P.alpha for i in range(0,len(A.grid.r))])
     
    bindFunction(d,'initialGasSurfaceDensity', initialGas)
    bindFunction(d,'gasSystole', updateGas)
    bindFunction(d,'alpha',updateAlpha)
    
    print('Re_starting simulation'+str(nameSim)+'..' )
    
    loadSimulation(d, dir=str(nameSim), files="data*.hdf5")
    d.initialize()
    d.evolve()
    
    """ Getting the variables needed for a contour plot"""
    
    with h5py.File(str(A.pars.outputDir) + '/data0050.hdf5') as fC: 
     rC = fC['grid/r'][()] 
     mC = fC['grid/m'][()] 
     sig_dC = fC['dust/Sigma'][()] 
     csC = fC['gas/cs'][()] 
     alphaC = fC['gas/alpha'][()] 
     VfragC = fC['dust/vFrag'][()] 
     SigmaC = fC["gas/Sigma"][()] 
     omegaC = fC["grid/OmegaK"][()] 
     St = fC["dust/St"][()] 
     d2g = fC["dust/dust2gasRatio"][()] 
     rIntC = fC["grid/rInt"][()] 
     rhoC = fC['dust/rhoBulk'][()][0,0] 
     agrain = fC['dust/a'][()][0] 
     Vk = omegaC * rC 

    FragBarrier = VfragC*VfragC / csC/csC/alphaC /3 
    FragBarrier *= 2 * SigmaC / np.pi /rhoC 
    p = SigmaC * omegaC * csC 
    
    """ The intensity derived from the dust surface denstity at the last snapshot is stored"""
    
    Ise1 = dens_to_int_v2(A.pars.outputDir,50,opacities = 'default_opacities.npz')
    
    R=Ise1[0]
    Int=Ise1[2] 
    
    np.savetxt(A.pars.outputDir+'/I_R_v1',(R,Int),delimiter=',', header="I,R",fmt='%1.8f')
    
    Ise2 = dens_to_int_v2(A.pars.outputDir,50,opacities = 'default_opacities_smooth.npz')
    
    R = Ise2[0]
    Int = Ise2[2] 
    
    np.savetxt(A.pars.outputDir+'/I_R_v2',(R,Int),delimiter=',', header="I,R",fmt='%1.8f')
    
    """ The variables neded to create a contour plot are also stored at each simulation folder, to see the contourplot just
    use the ContourPlot function in sim_funcions.py file"""

    _f = interp1d(np.log10(rC), np.log10(p), fill_value='extrapolate') 
    pInt = 10.**_f(np.log10(rIntC)) 
    Diff_pint = np.diff(pInt) 
    Diff_rint = np.diff(rIntC) 
    gammaC = np.abs(rC / p * Diff_pint / Diff_rint) 
    DriftBarrier = 2 * d2g * SigmaC * Vk[0]*Vk[0] /csC/csC/gammaC/ np.pi /rhoC 
    np.savetxt(str(A.pars.outputDir)+'/rC.txt',rC,delimiter = ',')
    np.savetxt(str(A.pars.outputDir)+'/agrain.txt',agrain,delimiter = ',')
    np.savetxt(str(A.pars.outputDir)+'/sigma.txt' ,np.log10(sig_dC).T,delimiter = ',')
    np.savetxt(str(A.pars.outputDir)+'/Frag.txt'  ,FragBarrier,delimiter = ',')
    np.savetxt(str(A.pars.outputDir)+'/Drift.txt' ,DriftBarrier,delimiter = ',')
      
    """ Movie of the Contour plot (population size vs position in arcsec)"""                           
    plotting.movie(str(A.pars.outputDir))


Make up a list of arguments that need to be processed

In [ ]:
Args = ([],[])

with Pool(argsin.cpus) as p:
        p.starmap(Restart_function,Args)


Reading from dump file: 'trial_24_3_15/dustpy.dmp'
Reading from dump file: 'trial_24_3_13/dustpy.dmp'
Reading from dump file: 'trial_24_3_14/dustpy.dmp'


63080000.0
Re_starting simulation trial_24_3_14..
63080000.0
Re_starting simulation trial_24_3_15..
63080000.0


/home/moon/bferrer/anaconda3/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


Restarting simulation from trial_24_3_14/data0001.hdf5
Re_starting simulation trial_24_3_13..


/home/moon/bferrer/anaconda3/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
/home/moon/bferrer/anaconda3/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


Restarting simulation from trial_24_3_15/data0001.hdf5
Restarting simulation from trial_24_3_13/data0001.hdf5


In [ ]:
""" A final message to know the simulations are done"""

print("All simulations complete!! ... it took long, but it was worth it!!")